# ibm_db.pconnect()

## Purpose:

Establish a persistent connection to an IBM Db2 server or database.

## Syntax:

`IBM_DBConnection ibm_db.pconnect( string `*`connectionInfo,`*` string `*`userID,`*` string `*`userPassword`*` [, dictionary `*`options`*` ] )`

## Parameters:

* __*connectionInfo*__&nbsp; &nbsp; : A valid Db2 database alias (if the database has been cataloged in the system database directory) *<u>or</u>* a connection string with the format `DRIVER={IBM DB2 ODBC DRIVER};ATTACH=connType;DATABASE=dbName;HOSTNAME=hostName;PORT=port;PROTOCOL=TCPIP;UID=userName;PWD=password` where:<p>
    
    * `connType`&nbsp; &nbsp; &nbsp;: Specifies whether the connection is to be made to a Db2 server or database; `TRUE` indicates the connection is to be made to a Db2 server and `FALSE` indicates the connection is to be made to a database.
    * `dbName`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The name of the Db2 server or database the connection is to be made to. *This option is only required when connecting to a Db2 database.*
    * `hostName`&nbsp; &nbsp; &nbsp;: The host name or IP address of the Db2 server the connection is to be made to. (The hostname is the name of the Db2 server, as it is known to the TCP/IP network.) *This option is only required when connecting to a Db2 server.*
    * `port`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The port number that receives Db2 connections on the server the connection is to be made to. (Port number __50000__ is used by default.) *This option is only required when connecting to a Db2 server.*
    * `userName`&nbsp; &nbsp; &nbsp;: The user name/ID that is to be used for authentication when the connection is established.
    * `password`&nbsp; &nbsp; &nbsp;: The password that corresponds to the user name/ID specified in the __*userName*__ parameter.<p>
               
* __*userID*__&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The user ID that is to be used for authentication when a connection is established. *If a Db2 database alias is NOT provided in the __connectionInfo__ parameter, this parameter must contain an empty string.*
* __*userPassword*__&nbsp; &nbsp; &nbsp; : The password that corresponds to the user ID specified in the __*userPassword*__ parameter. *If a Db2 database alias is NOT provided in the __connectionInfo__ parameter, this parameter must contain an empty string.* 
* __*options*__&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A dictionary containing key-value pairs for the attributes that are to be set before a connection is established. Valid keys for `IBM_DBConnection` objects are:<p>
    
    * `ibm_db.SQL_ATTR_AUTOCOMMIT`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: Specifies whether __AUTOCOMMIT__ mode is to be used or not;  `ibm_db.SQL_AUTOCOMMIT_ON` will turn __AUTOCOMMIT__ behavior __ON__ and `ibm_db.SQL_AUTOCOMMIT_OFF` will turn it __OFF__.
    * `ibm_db.SQL_ATTR_CASE`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The case that column names will be returned in; `ibm_db.CASE_NATURAL` will cause column names to be returned in natural case, `ibm_db.CASE_LOWER` will cause them to be returned in lower case, and `ibm_db.CASE_UPPER` will cause them to be returned in upper case. *(This attribute can only be used with Db2 databases on OS/390 or z/OS servers.)*    
    * `ibm_db.SQL_ATTR_CURRENT_SCHEMA`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The name of the schema to use, by default, when performing operations that require a schema name and one is not provided.
    * `ibm_db.SQL_ATTR_USE_TRUSTED_CONTEXT`&nbsp; &nbsp;: Specifies whether the connection to a Db2 server or database is to be treated as a trusted connection;  `ibm_db.SQL_TRUE` indicates the connection can be trusted and `ibm_db.SQL_FALSE` indicates the connection should not to be treated as a trusted connection.<p>

## Return values:

* If __successful__, a valid `IBM_DBConnection` object.
* If __unsuccessful__, the value `None`.

## Description:

The __ibm_db.pconnect()__ API is used to establish a persistent connection to an IBM Db2 server or database.<p>
    
Unlike connections that are established using the __ibm_db.connect()__ API, persistent connections created with the __ibm_db.pconnect()__ API are not closed with the __ibm_db.close()__ API. Instead, they are returned to a process-wide connection pool (when the __ibm_db.close()__ API is executed.) Then, the next time the __ibm_db.pconnect()__ API is called, the connection pool is searched for a matching connection and if one is found, it is returned to the application &mdash; new connections are not created unless there is no connection in the pool that meets the application's needs.<p>
    
As with the __ibm_db.connect()__ API, if a connection to a remote Db2 server or database is desired, the information needed to connect to the server (i.e., the host name, port number, and communications protocol) must be provided before a connection to either resource can be established. If a connection to a local, cataloged Db2 database is needed instead, just the alias for the database, as defined in the system database directory, must be supplied; values for the __*userID*__ and __*userPassword*__ parameters are optional. (If this API is called and just a local database alias is provided &mdash; for example, __conn = ibm_db.connect( 'SAMPLE', ' ', ' ' )__, &mdash; an attempt to establish a connection to the database will be made using the authorization ID and password for the current logged in Db2 instance user.)

## Examples:

### &nbsp; Example 1: Create a pool of 10 local Db2 database connections

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-pconnect_DB.py                                                             #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.pconnect() API to    #
#            create a pool of connections to a local Db2 database.                             #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.exec_immediate()                                                      #
#                 ibm_db.num_rows()                                                            #
#                 ibm_db.close()                                                               #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"        # The Alias For The Cataloged, Local Database
userID = "db2inst1"      # The Instance User ID At The Local Server
passWord = "Passw0rd"    # The Password For The Instance User ID At The Local Server
dbConnection = list(range(10))
resultSet = False
returnCode = False

# Construct The String That Will Be Used To Establish A Db2 Database Connection
connString = "ATTACH=FALSE"              # Attach To A Database; Not A Server
connString += ";DATABASE=" + dbName      # Required To Connect To A Database     
connString += ";PROTOCOL=TCPIP"
connString += ";UID=" + userID
connString += ";PWD=" + passWord

# Display A Status Message Indicating An Attempt To Establish Ten Connections To A Db2 
# Database Is About To Be Made
print("\nEstablishing 10 connections to the \'" + dbName + "\' database ... \n")

# Establish Ten Connections To The Local Db2 Database Specified
for loopCounter in range(10):

    # Attempt To Establish A Database Connection 
    try:
        dbConnection[loopCounter] = ibm_db.pconnect(connString, '', '')
    except Exception:
        pass

    # If A Connection Could Not Be Established, Display An Error Message And Continue
    if dbConnection[loopCounter] is None:
        print("\nERROR: Unable to connect to the \'" + dbName + "\' database.")
        continue

    # Otherwise, Display A "Connection Ready" Status Message
    else:
        print("  Connection {:>2} ready!" .format(loopCounter + 1))

# Add A Blank Line To The End Of The List Of Connections Created
print()

# Retrieve Data From The Database Using Connection Number Five
if not dbConnection[5] is None:

    # Define The SQL Statement That Is To Be Executed 
    sqlStatement = "SELECT * FROM department"

    # Set The Statement Option That Is To be Used When the Statement Is Executed
    stmtOption = {ibm_db.SQL_ATTR_ROWCOUNT_PREFETCH : ibm_db.SQL_ROWCOUNT_PREFETCH_ON}

    # Execute The SQL Statement Just Defined (Using The Desired Option)
    print("Executing the SQL statement \"" + sqlStatement + "\" from Connection 5 ... ", end="")
    currentConnection = dbConnection[5]
    try:
        resultSet = ibm_db.exec_immediate(currentConnection, sqlStatement, stmtOption)
    except Exception:
        pass

    # If The SQL Statement Could Not Be Executed, Display An Error Message And Continue
    if resultSet is False:
        print("\nERROR: Unable to execute the SQL statement specified.\n")

    # Otherwise, Complete The Status Message
    else:
        print("Done!\n")

    # Try To Find Out How Many Rows Are In Result Set That Was Produced By The Query Just
    # Executed (There Should Be 14 Rows)
    try:
        numRows = ibm_db.num_rows(resultSet)
    except Exception:
        pass
 
    # Display An Appropriate Message, Based On The Information Returned
    if numRows <= 0:
        print("Unable to obtain information about the number of rows returned.\n")
    else:
        print("Number of rows returned by the query: " + str(numRows) + "\n")

    # Attempt To Close The Database Connection (Connection 5)
    print("Closing database Connection 5 ... ", end="")
    try:
        returnCode = ibm_db.close(currentConnection)
    except Exception:
        pass

    # If The Connection Was Not Closed, Display An Error Message
    if returnCode is False:
        print("\nERROR: Unable to disconnect from the " + dbName + " database.")
        
    # Otherwise, Complete The Status Message
    else:
        print("Done!")
        print("(Connection 5 has been returned the pool of connections opened earlier.)\n")

# Display A Status Message Indicating An Attempt To Close The Remaining Db2 Database 
# Connections Is About To Be Made
print("Closing all remaining connections to the \'" + dbName + "\' database ... \n")

# Attempt To Close All Of The Db2 Database Connections That Were Opened Earlier
for loopCounter in range(10):
    
    # If The Specified Connection Is Open, Attempt To Close It
    if not dbConnection[loopCounter] is None:
        try:
            returnCode = ibm_db.close(dbConnection[loopCounter])
        except Exception:
            pass

        # If The Connection Could Not Be Closed, Display An Error Message And Continue
        if returnCode is False:
            print("\nERROR: Unable to disconnect from the " + dbName + " database.")
            continue

        # Otherwise, Display A "Connection Closed" Status Message
        else:
            print("  Connection {:>2} closed!" .format(loopCounter + 1))

# Display A Status Message Indicating All Database Connections Have Been Returned To The
# Connection Pool
print("\nAll database connections have been returned the pool of connections opened earlier.\n")

# Return Control To The Operating System
exit()


Establishing 10 connections to the 'SAMPLE' database ... 

  Connection  1 ready!
  Connection  2 ready!
  Connection  3 ready!
  Connection  4 ready!
  Connection  5 ready!
  Connection  6 ready!
  Connection  7 ready!
  Connection  8 ready!
  Connection  9 ready!
  Connection 10 ready!

Executing the SQL statement "SELECT * FROM department" from Connection 5 ... Done!

Number of rows returned by the query: 14

Closing database Connection 5 ... Done!
(Connection 5 has been returned the pool of connections opened earlier.)

Closing all remaining connections to the 'SAMPLE' database ... 

  Connection  1 closed!
  Connection  2 closed!
  Connection  3 closed!
  Connection  4 closed!
  Connection  5 closed!
  Connection  6 closed!
  Connection  7 closed!
  Connection  8 closed!
  Connection  9 closed!
  Connection 10 closed!

All database connections have been returned the pool of connections opened earlier.



### &nbsp; Restart the Jupyter Notebook to clear the Db2 connection pool<b>
    (This must be done before the second example can run)

In [ ]:
import os
os._exit(0)

### &nbsp; Example 2: Create a pool of 10 remote Db2 server connections

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-pconnect_SERVER.py                                                         #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.pconnect() API to    #
#            create a pool of connections to a remote Db2 server.                              #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.createdbNX()                                                          #
#                 ibm_db.close()                                                               #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
hostName = "197.126.80.22"    # IP Address Of Remote Server
portNum = "50000"             # Port Number That Receives Db2 Connections On The Remote Server 
userID = "db2inst2"           # The Instance User ID At The Remote Server
passWord = "ibmdb2"           # The Password For The Instance User ID At The Remote Server
svrConnection = list(range(10))
dbName = "MY_DB"
returnCode = None

# Construct The String That Will Be Used To Establish A Db2 Server Connection
connString = "DRIVER={IBM DB2 ODBC DRIVER}"
connString += ";ATTACH=TRUE"             # Attach To A Server; Not A Database
connString += ";DATABASE="               # Ignored When Connecting To A Server
connString += ";HOSTNAME=" + hostName    # Required To Connect To A Server
connString += ";PORT=" + portNum         # Required To Connect To A Server
connString += ";PROTOCOL=TCPIP"          # Required To Connect To A Server
connString += ";UID=" + userID
connString += ";PWD=" + passWord

# Display A Status Message Indicating An Attempt To Establish Ten Connections To A Remote 
# Db2 Server Is About To Be Made
print("\nEstablishing 10 connections to the \'" + hostName + "\' server ... \n")

# Establish Ten Connections To The Db2 Server Specified
for loopCounter in range(10):

    # Attempt To Establish A Db2 Server Connection 
    try:
        svrConnection[loopCounter] = ibm_db.pconnect(connString, '', '')
    except Exception:
        pass

    # If A Connection Could Not Be Established, Display An Error Message And Continue
    if svrConnection[loopCounter] is None:
        print("\nERROR: Unable to connect to the \'" + hostName + "\' server.")
        continue

    # Otherwise, Display A "Connection Ready" Status Message
    else:
        print("  Connection {:>2} ready!" .format(loopCounter + 1))

# Add A Blank Line To The End Of The List Of Connections Created
print()

# Attempt To Create A New Database At The Remote Server Using Connection Number Five
if not svrConnection[5] is None:
    print("Creating a database named " + dbName + " using Connection 5.  Please wait.")
    try:
        returnCode = ibm_db.createdbNX(svrConnection[5], dbName)
    except Exception:
        pass

    # If The Database Could Not Be Created, Display An Error Message And Exit 
    if returnCode is None:
        print("ERROR: Unable to create the " + dbName + " database.\n")
        errorMsg = ibm_db.conn_errormsg(svrConnection[5])
        print(errorMsg + "\n")

    # Otherwise, Display A Status Message Indicating The Database Was Created 
    else:
        print("\nThe database \"" + dbName + "\" has been created!\n")

    # Attempt To Close The Db2 Server Connection (Connection 5)
    print("Closing Db2 server Connection 5 ... ", end="")
    try:
        returnCode = ibm_db.close(svrConnection[5])
    except Exception:
        pass

    # If The Connection Was Not Closed, Display An Error Message
    if returnCode is False:
        print("\nERROR: Unable to disconnect from the " + hostName + " server.")
        
    # Otherwise, Complete The Status Message
    else:
        print("Done!")
        print("(Connection 5 has been returned the pool of connections opened earlier.)\n")

# Display A Status Message Indicating An Attempt To Close The Remaining Db2 Server
# Connections Is About To Be Made
print("Closing all remaining connections to the \'" + hostName + "\' server ... \n")

# Attempt To Close All Of The Remaining Db2 Server Connections That Were Opened Earlier
for loopCounter in range(10):
    
    # If The Specified Connection Is Open, Attempt To Close It
    if not svrConnection[loopCounter] is None:
        try:
            returnCode = ibm_db.close(svrConnection[loopCounter])
        except Exception:
            pass

        # If The Connection Could Not Be Closed, Display An Error Message And Continue
        if returnCode is False:
            print("\nERROR: Unable to disconnect from the " + hostName + " server.")
            continue

        # Otherwise, Display A "Connection Closed" Status Message
        else:
            print("  Connection {:>2} closed!" .format(loopCounter + 1))

# Display A Status Message Indicating All Db2 Server Connections Have Been Returned To The
# Connection Pool
print("\nAll Db2 server connections have been returned the pool of connections opened earlier.\n")

# Return Control To The Operating System
exit()


Establishing 10 connections to the '197.126.80.22' server ... 

  Connection  1 ready!
  Connection  2 ready!
  Connection  3 ready!
  Connection  4 ready!
  Connection  5 ready!
  Connection  6 ready!
  Connection  7 ready!
  Connection  8 ready!
  Connection  9 ready!
  Connection 10 ready!

Creating a database named MY_DB using Connection 5.  Please wait.

The database "MY_DB" has been created!

Closing Db2 server Connection 5 ... Done!
(Connection 5 has been returned the pool of connections opened earlier.)

Closing all remaining connections to the '197.126.80.22' server ... 

  Connection  1 closed!
  Connection  2 closed!
  Connection  3 closed!
  Connection  4 closed!
  Connection  5 closed!
  Connection  6 closed!
  Connection  7 closed!
  Connection  8 closed!
  Connection  9 closed!
  Connection 10 closed!

All Db2 server connections have been returned the pool of connections opened earlier.

